In [1]:
from src.constants.constants import *
from src.feature_extraction.extract_features import *
from src.utils.helper import *
from src.models.evaluate import *
from src.models.train import *
from src.preprocessing.audio_cleaning import *
import warnings

warnings.filterwarnings("ignore")


In [2]:
N_NEIGHBORS

5

In [3]:
import os
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from src.utils.helper import get_splitted_data


In [41]:
data = pd.read_csv('csv_files/data.csv')
features = pd.read_csv('csv_files/features.csv')
df = pd.merge(features, 
              data.drop(columns=['client_id', 'sentence', 'up_votes', 'down_votes', 'accent']), 
              left_on='voice_id', right_on='path', how='inner')
df.drop(columns=['path', 'Unnamed: 0'], inplace=True)
df.head()

,mean_mfcc_0,mean_mfcc_1,mean_mfcc_2,mean_mfcc_3,mean_mfcc_4,mean_mfcc_5,mean_mfcc_6,mean_mfcc_7,mean_mfcc_8,mean_mfcc_9,...,spectral_contrast_4,spectral_contrast_5,spectral_contrast_6,zcr,rms,pitch,voice_id,age,gender,label
0,-403.220062,99.762260,20.534065,41.544975,-7.227066,31.042414,-2.121207,-0.940516,2.747488,7.995158,...,17.295187,18.791175,52.073479,0.171368,0.034952,1656.884644,common_voice_en_181169.mp3,fifties,male,2
1,-223.813477,157.473358,11.694227,66.153183,-23.514465,0.811838,-19.525723,-8.694965,-6.325938,-10.528963,...,16.227987,21.994841,50.559168,0.114894,0.167097,1285.268188,common_voice_en_122004.mp3,twenties,male,0
2,-415.865570,126.014847,-36.185616,48.158478,-13.574215,-18.600719,-39.063217,-24.592718,-17.863024,-13.136163,...,21.419817,25.141222,50.535503,0.097565,0.054672,1702.509766,common_voice_en_10208833.mp3,fifties,female,3
3,-423.244720,198.545227,12.053857,62.119160,17.754673,-21.133663,6.490955,-1.768311,-28.847279,-15.727254,...,20.016475,19.788320,47.486808,0.041022,0.126251,820.089233,common_voice_en_18618243.mp3,twenties,male,0
4,-223.693176,74.879898,9.623554,114.356293,-12.664084,-20.089724,-38.216282,-0.664027,-42.319389,-13.502878,...,19.183877,25.547451,51.613073,0.188517,0.143542,1860.538818,common_voice_en_18075517.mp3,twenties,male,0


In [42]:
X = df.drop(columns=['label', 'gender', 'age', 'voice_id'])
y = df['label']

X_val_train, X_test, y_val_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42, stratify=y)
X_train, X_val, y_train, y_val = train_test_split(X_val_train, y_val_train, test_size=0.18, random_state=42, stratify=y_val_train)

In [43]:
Xa_train, Xa_val, ya_train, ya_val = get_splitted_data(X, y, df, 'age')
Xg_train, Xg_val, yg_train, yg_val = get_splitted_data(X, y, df, 'gender')

In [62]:
model_a = load_model("src/selected_model/knn_age.pkl")
lgbm_model_g = load_model("src/selected_model/gender_lgbm_model.pkl")
knn_model_g = load_model("src/selected_model/knn_gender.pkl")
model =  load_model("src/selected_model/knn_label.pkl")

In [49]:
%%time
evaluate_multi(model, X_test, y_test, type = 2)



TESTING RESULTS: 
CONFUSION MATRIX:
[[17504   174   185    92]
 [  125  2608     4   164]
 [  394    15  2176     9]
 [  119   215     4  2036]]
ACCURACY SCORE:
0.9419
CPU times: user 1min 14s, sys: 2.16 s, total: 1min 16s
Wall time: 9.37 s


In [50]:
x = pd.merge(df, X_test, left_index=True, right_index=True, how='right')
x.head()

,mean_mfcc_0_x,mean_mfcc_1_x,mean_mfcc_2_x,mean_mfcc_3_x,mean_mfcc_4_x,mean_mfcc_5_x,mean_mfcc_6_x,mean_mfcc_7_x,mean_mfcc_8_x,mean_mfcc_9_x,...,spectral_contrast_0_y,spectral_contrast_1_y,spectral_contrast_2_y,spectral_contrast_3_y,spectral_contrast_4_y,spectral_contrast_5_y,spectral_contrast_6_y,zcr_y,rms_y,pitch_y
87069,-281.944061,201.146835,4.391376,68.258842,-17.089682,6.560512,-5.857045,-28.403639,-3.634766,-4.248062,...,22.934863,15.511947,19.644486,15.751045,15.643727,24.349948,47.090268,0.062500,0.137512,669.412964
42665,-308.221893,173.087036,0.801586,-28.245972,-64.956612,-32.546398,-15.978315,-3.659044,-15.762386,-12.324957,...,19.037665,21.852489,22.638533,23.952031,21.982025,25.577788,58.544866,0.087782,0.169417,696.325500
109016,-303.039154,150.930984,-47.686619,53.087147,-26.411861,-18.253426,-18.209721,-20.466351,-14.088178,-14.212386,...,24.754715,20.383918,22.112408,19.775835,20.115320,25.423014,54.250347,0.134141,0.118296,1473.150513
110402,-433.801331,72.147812,7.029837,43.468819,-16.971661,-17.044973,-37.087997,-11.829974,-15.852037,-0.625612,...,18.817950,14.910822,19.866912,15.476118,17.182655,20.422225,55.155081,0.249068,0.028486,1789.121826
80457,-260.183167,109.363174,33.693771,52.015930,14.755984,-24.705837,3.136095,4.209895,-19.941381,6.267535,...,21.759407,15.643544,19.149491,17.935279,17.806459,21.008033,52.115119,0.184491,0.145807,1409.390747


In [63]:
ya_pred = model_a.predict(X_test)
lgbm_yg_pred = lgbm_model_g.predict(X_test)
knn_yg_pred = knn_model_g.predict(X_test)

In [58]:
xa = x['age']
lea = LabelEncoder()
xa = lea.fit_transform(xa)
acca_train = accuracy_score(xa, ya_pred)
print("Accuracy for age: ", acca_train)

Accuracy for age:  0.9674721189591078


In [64]:
xg = x['gender']
leg = LabelEncoder()
xg = leg.fit_transform(xg)

lgbm_accg_train = accuracy_score(xg, lgbm_yg_pred)
knn_accg_train = accuracy_score(xg, knn_yg_pred)

print("Accuracy for gender LGBM: ", lgbm_accg_train)
print("Accuracy for gender KNN: ", knn_accg_train)

Accuracy for gender LGBM:  0.9949271995043371
Accuracy for gender KNN:  0.9838522304832714


In [65]:
lgbm_combined = 3 - (1*lgbm_yg_pred + 2*ya_pred)
knn_combined = 3 - (1*knn_yg_pred + 2*ya_pred)

In [66]:
y_pred = model.predict(X_test)

acc_lgbm = accuracy_score(lgbm_combined, y_pred)
acc_knn = accuracy_score(knn_combined, y_pred)
print("Final Accuracy LGBM", acc_lgbm)
print("Final Accuracy KNN", acc_knn)

Final Accuracy LGBM 0.9555065055762082
Final Accuracy KNN 0.9656908302354399
